In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26912831
paper_name = 'saka_kobayashi_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/geldata_201604042348.csv')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5153 x 9


In [7]:
original_data.head()

,number,ORF name,Gene name,rDNA stability,rDNA copy number,Gel(Plate/Row/Colum),Replicative lifespan,Other chromosome abnormality,InputDate
0,1,YAL068C,PAU8,2,4,1-A-2,NaN,NaN,0000-00-00 00:00:00
1,2,YAL067C,SEO1,2,2,1-A-3,NaN,NaN,0000-00-00 00:00:00
2,3,YAL066W,YAL066W,2,2,1-A-4,NaN,NaN,0000-00-00 00:00:00
3,4,YAL065C,YAL065C,2,2,1-A-5,NaN,NaN,0000-00-00 00:00:00
4,5,YAL062W,GDH3,2,2,1-A-6,NaN,NaN,0000-00-00 00:00:00


In [8]:
original_data.loc[original_data['ORF name']=='YAL055W',:]

,number,ORF name,Gene name,rDNA stability,rDNA copy number,Gel(Plate/Row/Colum),Replicative lifespan,Other chromosome abnormality,InputDate
10,11,YAL055W,PEX22,3,2,1-A-12,NaN,NaN,0000-00-00 00:00:00


In [9]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [number, ORF name, Gene name, rDNA stability, rDNA copy number, Gel(Plate/Row/Colum), Replicative lifespan, Other chromosome abnormality, InputDate, orf]
Index: []


In [13]:
original_data['data1'] = pd.to_numeric(original_data['rDNA stability'], errors='coerce')

In [14]:
def convert_and_average(s):
    lst = s.split(',')
    lst2 = []
    for t in lst:
        try:
            v = int(t)
        except ValueError:
            v = np.nan
        lst2.append(v)
    mn = np.nanmean(np.array(lst2))
    return mn

In [15]:
original_data['data2'] = original_data['rDNA copy number'].apply(convert_and_average)

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


In [16]:
# wt = 0, h = negative values
original_data['data1'] = 2-original_data['data1']

In [17]:
# wt = 0, s = positive values
original_data['data2'] = original_data['data2']-2

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data1','data2']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4943, 2)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [756, 746]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,756,746
data_type,value,value
orf,,
YAL002W,0.0,0.0
YAL004W,0.0,0.0
YAL005C,0.0,0.0
YAL007C,-1.0,0.0
YAL008W,0.0,0.0


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [28]:
data.head()

,dataset_id,756,746
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.0,0.0
1863,YAL004W,0.0,0.0
4,YAL005C,0.0,0.0
5,YAL007C,-1.0,0.0
6,YAL008W,0.0,0.0


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [32]:
data_all.head()


,dataset_id,756,746,756,746
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.0,0.0,0.000000,0.0
1863,YAL004W,0.0,0.0,0.000000,0.0
4,YAL005C,0.0,0.0,0.000000,0.0
5,YAL007C,-1.0,0.0,-2.438213,0.0
6,YAL008W,0.0,0.0,0.000000,0.0


# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 26912831...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.84s/it]

Updating the data_modified_on field...
